In [1]:
# 토크나이져
import tensorflow as tf
Tokenizer = tf.keras.preprocessing.text.Tokenizer
pad_sequences = tf.keras.preprocessing.sequence.pad_sequences

2024-11-01 17:57:01.735296: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-01 17:57:01.799611: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-01 17:57:02.659463: I tensorflow/c/logging.cc:34] Successfully opened dynamic library libdirectml.d6f03b303ac3c4f2eeb8ca631688c9757b361310.so
2024-11-01 17:57:02.659560: I tensorflow/c/logging.cc:34] Successfully opened dynamic library libdxcore.so
2024-11-01 17:57:02.663133: I tensorflow/c/logging.cc:34] Successfully opened dynamic library libd

In [2]:
sentences = ['이 영화는 재미있어요',
             '배우들의 연기가 훌륭하네요',
             '스토리가 흥미로워서 시간가는 줄 몰랐어요']

In [3]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(sentences)

In [4]:
# 단어 인덱스
word_index = tokenizer.word_index
word_index

{'이': 1,
 '영화는': 2,
 '재미있어요': 3,
 '배우들의': 4,
 '연기가': 5,
 '훌륭하네요': 6,
 '스토리가': 7,
 '흥미로워서': 8,
 '시간가는': 9,
 '줄': 10,
 '몰랐어요': 11}

In [5]:
# 정수시퀀스
sequence =  tokenizer.texts_to_sequences(sentences)
sequence

[[1, 2, 3], [4, 5, 6], [7, 8, 9, 10, 11]]

In [6]:
# pad_sequence 사용  -- 길이를 맞춰준다
sequence = [
    [1,2,3],
    [1,1],
    [1,2,34,5]
]
# 길이를 3개로 맞추자
pad_sequences(sequence, maxlen=3,padding='post')  # 모자라면뒤에서 채운다. 길면 뒤에서부터 카운트해서 앞의 글자를자른다

array([[ 1,  2,  3],
       [ 1,  1,  0],
       [ 2, 34,  5]], dtype=int32)

In [7]:
# 허깅페이스 토크나이져
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
encoded_input = tokenizer(sentences, padding=True, truncation=True,max_length=5
                          # , return_tensors='tf'
                          , return_tensors='pt'
                          )
# 인코딩된 데이터
# encoded_input
# 디코딩 데이터
tokenizer.decode(encoded_input['input_ids'][1])

'[CLS] 배우들의 연 [SEP]'

In [8]:
# 다운로드
import urllib
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt", filename="ratings_train.txt")
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt", filename="ratings_test.txt")

('ratings_test.txt', <http.client.HTTPMessage at 0x7f55c04002b0>)

In [9]:
from transformers import BertTokenizer
import pandas as pd
from sklearn.model_selection import train_test_split
# 전이학습 파인튜닝
# 데이터 로드  **
train_data = pd.read_table('ratings_train.txt')[['document',  'label']]
test_data = pd.read_table('ratings_test.txt')[['document',  'label']]
# 결측치 제거
train_data = train_data.dropna()
test_data = test_data.dropna()
# 한글만 정규식으로 추출
import re
train_data['document'] = train_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
test_data['document'] = test_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
# 데이터셋 분할 train, val
train_texts, val_texts, train_labels, val_labels = train_test_split(
    train_data['document'].tolist(), train_data['label'].tolist(), test_size=0.2, random_state=42
    )
# 버트토크나이져 로드  **
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased',clean_up_tokenization_spaces=True)
# BERT 입력형식으로 인코딩  **
train_encoder = tokenizer(train_texts, padding=True, truncation=True, max_length=128)
val_encoder = tokenizer(val_texts, padding=True, truncation=True, max_length=128)

/tmp/ipykernel_13090/4004566949.py:13: FutureWarning: The default value of regex will change from True to False in a future version.
  train_data['document'] = train_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
/tmp/ipykernel_13090/4004566949.py:14: FutureWarning: The default value of regex will change from True to False in a future version.
  test_data['document'] = test_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")


In [10]:
# 인코딩된 데이터를파이토치 데이터셋으로  *
import torch
class BertDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

train_dataset = BertDataset(train_encoder, train_labels)
val_dataset = BertDataset(val_encoder, val_labels)
# BERT 모델 로드(전이학습) *
from transformers import BertForSequenceClassification
model = BertForSequenceClassification.from_pretrained('bert-base-multilingual-cased', num_labels=2)
# 학습(파인튜닝) *
from transformers import Trainer, TrainingArguments
# 학습인자설정
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=16,
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    report_to='none'  # W&B 비활성
)
# trainer 초기화
trainer =  Trainer(model = model, args = training_args
        , train_dataset=train_dataset, eval_dataset = val_dataset)


/home/sangho/miniconda3/envs/ml3/lib/python3.9/site-packages/torchvision/datapoints/__init__.py:12: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https://github.com/pytorch/vision/issues/6753, and you can also check out https://github.com/pytorch/vision/issues/7319 to learn more about the APIs that we suspect might involve future changes. You can silence this warning by calling torchvision.disable_beta_transforms_warning().
  warnings.warn(_BETA_TRANSFORMS_WARNING)
/home/sangho/miniconda3/envs/ml3/lib/python3.9/site-packages/torchvision/transforms/v2/__init__.py:54: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any fe

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
# 모델파인튜닝
trainer.train()

AttributeError: 'AdamW' object has no attribute 'train'

In [ ]:
trainer.evaluate()
#모델 저장
model.save_pretrained('./results')
tokenizer.save_pretrained('./results')